<a href="https://colab.research.google.com/github/yukinaga/langchain/blob/main/section_4/03_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# デプロイの準備
Streamlit Community Cloudにアプリをデプロイする準備をします。

## 翻訳アプリのコード
`%%writefile`のマジックコマンドを使って、翻訳アプリのコードを「app.py」に書き込みます。  
今回、OpneAIのAPI keyは、Streamlit Community Cloudの「Secrets」に保存します。  

In [ ]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
import os

# Streamlit Community Cloudの「Secrets」からOpenAI API keyを取得
os.environ["OPENAI_API_KEY"] = st.secrets.OpenAIAPI.openai_api_key

chat = ChatOpenAI(model="gpt-3.5-turbo")

# プロンプトのテンプレート
system_template = (
    "あなたは、{source_lang} を {target_lang}に翻訳する優秀な翻訳アシスタントです。翻訳結果以外は出力しないでください。"
)
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

if "response" not in st.session_state:
    st.session_state["response"]= ""

# LLMとやりとりする関数
def communicate():
    text = st.session_state["user_input"]
    response = chat(
        chat_prompt.format_prompt(
            source_lang=source_lang, target_lang=target_lang, text=text
        ).to_messages()
    )
    st.session_state["response"] = response.content

# ユーザーインターフェイスの構築
st.title("翻訳アプリ")
st.write("LangChainを使った翻訳アプリです。")

options = ["日本語", "英語", "スペイン語", "ドイツ語", "フランス語", "中国語"]
source_lang = st.selectbox(label="翻訳元", options=options)
target_lang = st.selectbox(label="翻訳先", options=options)
st.text_input("翻訳する文章を入力してください。", key="user_input")
st.button("翻訳", type="primary", on_click=communicate)

if st.session_state["user_input"] != "":
    st.write("翻訳結果:")
    st.write(st.session_state["response"])

## requirements.txtの作成
Streamlit Community Cloudのサーバー上でアプリを動かすために、「requirements.txt」を作成する必要があります。  
このファイルでは、必要なライブラリを指定します。  
以下のセルでは、`%%writefile`のマジックコマンドを使って、必要なライブラリを「requirements.txt」に書き込みます。  

In [ ]:
%%writefile requirements.txt
streamlit==1.20.0
langchain
langchain-openai

以下の作成されたファイルをダウンロードして、GitHubのレポジトリにアップしましょう。
* app.py
* requirements.txt